In [1]:
import torch
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler, random_split
import pandas as pd
import numpy as np
import os
import random
from collections import defaultdict, Counter
from itertools import combinations
import json
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import KFold
import optuna
from torchmetrics.classification import F1Score
import pickle

import sys
### import Dataset prepartion and model training classes from BS_LS_scripts folder
sys.path.insert(1, '/home/wangc90/circRNA/circRNA_Data/BS_LS_scripts/')
from BS_LS_DataSet import BS_LS_DataSet_Prep, RCM_Score
from BS_LS_Training_Base_models_0 import Objective, Objective_CV


In [2]:
class RCM_optuna_flanking(nn.Module):
    '''
        This is for 2-d model to process the RCM score distribution of the flanking introns
    '''
    def __init__(self, trial):
        
        super(RCM_optuna_flanking, self).__init__()
        
        # convlayer 1
        self.out_channel1 = trial.suggest_categorical('flanking_out_channel1', [128, 256, 512])
#         self.out_channel1 = 128

#         kernel_size1 = 5

        self.conv1 = nn.Conv1d(in_channels=5, out_channels=self.out_channel1,\
                               kernel_size=5, stride=5, padding=0)
        self.conv1_bn = nn.BatchNorm1d(self.out_channel1)
        
        self.out_channel2 = trial.suggest_categorical('flanking_out_channel2', [128, 256, 512])
#         self.out_channel2 = 32
        
        self.conv2 = nn.Conv1d(in_channels=self.out_channel1, out_channels=self.out_channel2,\
                               kernel_size=5, stride=5, padding=0)
        
        self.conv2_bn = nn.BatchNorm1d(self.out_channel2)
        
        self.conv2_out_dim = 5
    

    def forward(self, x):
        out = x
        out = torch.relu(self.conv1_bn(self.conv1(out)))
        out = torch.relu(self.conv2_bn(self.conv2(out)))

        out = out.view(-1, self.out_channel2 * self.conv2_out_dim)
        return out

    
class RCM_optuna_upper(nn.Module):
    '''
        This is for 2-d model to process the RCM score distribution of the upper introns
    '''
    def __init__(self, trial):
        
        super(RCM_optuna_upper, self).__init__()
        
        # convlayer 1
        self.out_channel1 = trial.suggest_categorical('upper_out_channel1', [128, 256, 512])
#         self.out_channel1 = 512

        self.conv1 = nn.Conv1d(in_channels=5, out_channels=self.out_channel1,\
                               kernel_size=5, stride=5, padding=0)
        
        self.conv1_bn = nn.BatchNorm1d(self.out_channel1)
        
        self.out_channel2 = trial.suggest_categorical('upper_out_channel2', [128, 256, 512])
#         self.out_channel2 = 64
        
        self.conv2 = nn.Conv1d(in_channels=self.out_channel1, out_channels=self.out_channel2,\
                               kernel_size=5, stride=5, padding=0)
        
        self.conv2_bn = nn.BatchNorm1d(self.out_channel2)
        self.conv2_out_dim = 5
    

    def forward(self, x):
        out = x
        out = torch.relu(self.conv1_bn(self.conv1(out)))
        out = torch.relu(self.conv2_bn(self.conv2(out)))

        out = out.view(-1, self.out_channel2 * self.conv2_out_dim)
        return out
    
    
class RCM_optuna_lower(nn.Module):
    '''
        This is for 2-d model to process the RCM score distribution of the lower introns
    '''
    def __init__(self, trial):
        
        super(RCM_optuna_lower, self).__init__()
        
        # convlayer 1
        self.out_channel1 = trial.suggest_categorical('lower_out_channel1', [128, 256, 512])
#         self.out_channel1 = 512

        self.conv1 = nn.Conv1d(in_channels=5, out_channels=self.out_channel1,\
                               kernel_size=5, stride=5, padding=0)
        
        self.conv1_bn = nn.BatchNorm1d(self.out_channel1)
        self.out_channel2 = trial.suggest_categorical('lower_out_channel2', [128, 256, 512])
#         self.out_channel2 = 512
        
        self.conv2 = nn.Conv1d(in_channels=self.out_channel1, out_channels=self.out_channel2,\
                               kernel_size=5, stride=5, padding=0)
        
        self.conv2_bn = nn.BatchNorm1d(self.out_channel2)
        self.conv2_out_dim = 5
    

    def forward(self, x):
        out = x
        out = torch.relu(self.conv1_bn(self.conv1(out)))
        out = torch.relu(self.conv2_bn(self.conv2(out)))

        out = out.view(-1, self.out_channel2 * self.conv2_out_dim)
        return out
    
    

class RCM_optuna_concate(nn.Module):
    ''''
        
    '''
    def __init__(self, trial):
        
        super(RCM_optuna_concate, self).__init__()

        ### cnn for the flanking rcm scores
        self.cnn_flanking = RCM_optuna_flanking(trial)

        self.flanking_out_dim = self.cnn_flanking.conv2_out_dim
        self.flanking_out_channel = self.cnn_flanking.out_channel2
#         print(f'flanking out dim: {self.flanking_out_dim}, flanking out channel {self.flanking_out_channel}')
        
        ### cnn for the upper rcm scores
        self.cnn_upper = RCM_optuna_upper(trial)

        self.upper_out_dim = self.cnn_upper.conv2_out_dim
        self.upper_out_channel = self.cnn_upper.out_channel2
#         print(f'upper_out_dim: {self.upper_out_dim}, upper_out_channel {self.upper_out_channel}')
        
        ### cnn for the lower rcm scores
        self.cnn_lower = RCM_optuna_lower(trial)

        self.lower_out_dim = self.cnn_lower.conv2_out_dim
        self.lower_out_channel = self.cnn_lower.out_channel2
#         print(f'lower_out_dim: {self.lower_out_dim}, lower_out_channel {self.lower_out_channel}')
        

        self.fc1_input_dim = self.flanking_out_dim * self.flanking_out_channel + \
                             self.upper_out_dim * self.upper_out_channel + \
                             self.lower_out_dim * self.lower_out_channel

#         print(f'fc1_input_dim: {self.fc1_input_dim}')
        
        
        self.fc1_out = trial.suggest_categorical('concat_fc1_out', [128, 256, 512])
#         self.fc1_out = 512
    
        # add the rcm feature dimension here as well (5*5+2)*3+2 = 83
        self.fc1 = nn.Linear(self.fc1_input_dim, self.fc1_out)
        
        self.fc1_bn = nn.BatchNorm1d(self.fc1_out)

        dropout_rate_fc1 = trial.suggest_categorical("concat_dropout_rate_fc1",  [0, 0.1, 0.2, 0.4])
        self.drop_nn1 = nn.Dropout(p=dropout_rate_fc1)

        # fc layer2
        # use dimension output with nn.CrossEntropyLoss()
        self.fc2_out = trial.suggest_categorical('concat_fc2_out', [4, 8, 16, 32])
#         self.fc2_out = 8
        self.fc2 = nn.Linear(self.fc1_out, self.fc2_out)

        self.fc2_bn = nn.BatchNorm1d(self.fc2_out)

        dropout_rate_fc2 = trial.suggest_categorical("concat_dropout_rate_fc2",[0, 0.1, 0.2, 0.4])
    
        self.drop_nn2 = nn.Dropout(p=dropout_rate_fc2)

        self.fc3 = nn.Linear(self.fc2_out, 2)
        

    def forward(self, rcm_flanking, rcm_upper, rcm_lower):
        
        x1 = self.cnn_flanking(rcm_flanking)

        x2 = self.cnn_upper(rcm_upper)
        
        x3 = self.cnn_lower(rcm_lower)
        
        x = torch.cat((x1,x2,x3), dim=1)
    
        # feed the concatenated feature to fc1
        out = self.fc1(x)
        out = self.drop_nn1(torch.relu(self.fc1_bn(out)))
        out = self.fc2(out)
        out = self.drop_nn2(torch.relu(self.fc2_bn(out)))
        out = self.fc3(out)
        return out


In [3]:
def rcm_flankingWithin_small_windows_optuna(num_trial):
    
    ## specify different kmer length to get the training data of the rcm score for that kmer 
    ### just change this number to 10, 20, 40 and 80 to get the model performance for different kmer length

    study = optuna.create_study(direction='maximize')

    ### where to save the 3-fold CV validation acc based on the rcm score and mlp

    val_acc_folder = f'/home/wangc90/circRNA/circRNA_Data/model_outputs/rcm_flankingWithin_small_windows/9000/val_acc_cv3'
    ### wehre to save the detailed optuna results
    optuna_folder = f'/home/wangc90/circRNA/circRNA_Data/model_outputs/rcm_flankingWithin_small_windows/9000/optuna'
    
    
    BS_LS_coordinates_path = '/home/wangc90/circRNA/circRNA_Data/BS_LS_data/updated_data/BS_LS_coordinates_final.csv'
    hg19_seq_dict_json_path = '/home/wangc90/circRNA/circRNA_Data/hg19_seq/hg19_seq_dict.json'
    flanking_dict_folder = '/home/wangc90/circRNA/circRNA_Data/BS_LS_data/flanking_dicts/'
    bs_ls_dataset = BS_LS_DataSet_Prep(BS_LS_coordinates_path=BS_LS_coordinates_path,
                                   hg19_seq_dict_json_path=hg19_seq_dict_json_path,
                                   flanking_dict_folder=flanking_dict_folder,
                                   flanking_junction_bps=100,
                                   flanking_intron_bps=5000,
                                   training_size=9000)


    ## generate the junction and flanking intron dict
    bs_ls_dataset.get_junction_flanking_intron_seq()

    train_key_1, _, test_keys = bs_ls_dataset.get_train_test_keys()

    rcm_scores_folder = '/home/wangc90/circRNA/circRNA_Data/BS_LS_data/flanking_dicts/rcm_scores/'

    ### try with rcm features
    _, _, train_torch_flanking_rcm, train_torch_upper_rcm,\
    train_torch_lower_rcm, train_torch_labels = bs_ls_dataset.seq_to_tensor(data_keys=train_key_1,\
                                                                            rcm_folder=rcm_scores_folder,\
                                                                            is_rcm=True,\
                                                                            is_upper_lower_concat=False)
    
#     print(train_torch_flanking_rcm.shape)

    RCM_kmer_Score_dataset = RCM_Score(flanking_only=False,
                                       flanking_rcm=train_torch_flanking_rcm,\
                                       upper_rcm=train_torch_upper_rcm,\
                                       lower_rcm=train_torch_lower_rcm,\
                                       label=train_torch_labels)
    
    
    print(len(RCM_kmer_Score_dataset))
    
    study = optuna.create_study(pruner=optuna.pruners.MedianPruner(n_warmup_steps=1, n_startup_trials=10),
                                direction='maximize')


    study.optimize(Objective_CV(cv=3, model= RCM_optuna_concate, 
                                dataset=RCM_kmer_Score_dataset,
                                val_acc_folder=val_acc_folder), n_trials=num_trial, gc_after_trial=True)


    pruned_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.PRUNED]
    complete_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE]
    with open(optuna_folder+'/optuna.txt', 'a') as f:
        f.write("Study statistiqcs: \n")
        f.write(f"Number of finished trials: {len(study.trials)}\n")
        f.write(f"Number of pruned trials: {len(pruned_trials)}\n")
        f.write(f"Number of complete trials: {len(complete_trials)}\n")

        f.write("Best trial:\n")
        trial = study.best_trial
        f.write(f"Value: {trial.value}\n")
        f.write("Params:\n")
        for key, value in trial.params.items():
            f.write(f"{key}:{value}\n")

    df = study.trials_dataframe().drop(['state','datetime_start','datetime_complete','duration','number'], axis=1)
    df.to_csv(optuna_folder + '/optuna.csv', sep='\t', index=None)

In [ ]:
rcm_flankingWithin_small_windows_optuna(num_trial=500)

[I 2023-11-22 19:16:10,265] A new study created in memory with name: no-name-d5202fcd-6402-478c-8690-052784fd2582


chr5|138837130|138837392|- has N in the extracted junctions, belongs to BS
There are 0 overlapped flanking sequence from BS and LS  
There are 7 repeated BS sequences
There are 2 repeated LS sequences


[I 2023-11-22 19:18:25,015] A new study created in memory with name: no-name-ca7b90a1-7a22-49e8-957d-fca2926808d5


18000
fold 1, epoch 20, val loss 8.767366170883179 val accuracy 0.6003
fold 1, epoch 40, val loss 9.889778137207031 val accuracy 0.5938
fold 1, epoch 60, val loss 11.187600076198578 val accuracy 0.605
fold 1, epoch 80, val loss 12.18494176864624 val accuracy 0.6042
fold 1, epoch 100, val loss 13.006317138671875 val accuracy 0.6037
fold 1, epoch 120, val loss 14.115652084350586 val accuracy 0.6115
fold 2, epoch 20, val loss 8.459560692310333 val accuracy 0.5913
fold 2, epoch 40, val loss 9.164281249046326 val accuracy 0.6097
fold 2, epoch 60, val loss 10.19625872373581 val accuracy 0.6105
fold 2, epoch 80, val loss 11.65113478899002 val accuracy 0.6088
fold 2, epoch 100, val loss 12.778429448604584 val accuracy 0.6045
fold 2, epoch 120, val loss 13.454310774803162 val accuracy 0.611
fold 3, epoch 20, val loss 8.369351089000702 val accuracy 0.599
fold 3, epoch 40, val loss 9.264171540737152 val accuracy 0.6073
fold 3, epoch 60, val loss 10.234699130058289 val accuracy 0.6147
fold 3, epoc

[I 2023-11-22 19:33:53,245] Trial 0 finished with value: 0.6095 and parameters: {'lr': 0.0002707952072734182, 'l2_lambda': 2.7527204568094844e-08, 'batch_size': 512, 'epochs': 120, 'flanking_out_channel1': 128, 'flanking_out_channel2': 256, 'upper_out_channel1': 256, 'upper_out_channel2': 256, 'lower_out_channel1': 128, 'lower_out_channel2': 256, 'concat_fc1_out': 256, 'concat_dropout_rate_fc1': 0.1, 'concat_fc2_out': 4, 'concat_dropout_rate_fc2': 0.1}. Best is trial 0 with value: 0.6095.


...

fold 1, epoch 20, val loss 16.149396121501923 val accuracy 0.6317
fold 1, epoch 40, val loss 16.649711668491364 val accuracy 0.6427
fold 1, epoch 60, val loss 17.839934170246124 val accuracy 0.6542
fold 1, epoch 80, val loss 19.328479409217834 val accuracy 0.6488
fold 1, epoch 100, val loss 21.233786523342133 val accuracy 0.6518
fold 1, epoch 120, val loss 22.587694883346558 val accuracy 0.656
fold 2, epoch 20, val loss 15.816957175731659 val accuracy 0.6382
fold 2, epoch 40, val loss 15.899355947971344 val accuracy 0.6475
fold 2, epoch 60, val loss 16.93992990255356 val accuracy 0.6495
fold 2, epoch 80, val loss 18.384577929973602 val accuracy 0.6562
fold 2, epoch 100, val loss 20.291907966136932 val accuracy 0.6532
fold 2, epoch 120, val loss 21.904040694236755 val accuracy 0.6497
fold 3, epoch 20, val loss 15.428453207015991 val accuracy 0.6323
fold 3, epoch 40, val loss 15.60856419801712 val accuracy 0.6473
fold 3, epoch 60, val loss 15.795445322990417 val accuracy 0.6572
fold 3, e

[I 2023-11-22 21:14:25,268] Trial 5 finished with value: 0.6523 and parameters: {'lr': 6.180965726328772e-05, 'l2_lambda': 1.3313985083188563e-09, 'batch_size': 256, 'epochs': 120, 'flanking_out_channel1': 128, 'flanking_out_channel2': 128, 'upper_out_channel1': 512, 'upper_out_channel2': 256, 'lower_out_channel1': 512, 'lower_out_channel2': 512, 'concat_fc1_out': 512, 'concat_dropout_rate_fc1': 0.1, 'concat_fc2_out': 8, 'concat_dropout_rate_fc2': 0.2}. Best is trial 5 with value: 0.6523.


fold 3, epoch 120, val loss 20.20127445459366 val accuracy 0.6512
fold 1, epoch 20, val loss 142.39729177951813 val accuracy 0.5817
fold 1, epoch 40, val loss 125.321537733078 val accuracy 0.5908
fold 1, epoch 60, val loss 162.7306448817253 val accuracy 0.5965
fold 1, epoch 80, val loss 166.13279259204865 val accuracy 0.5918
fold 1, epoch 100, val loss 178.8334949016571 val accuracy 0.596
fold 1, epoch 120, val loss 186.93602204322815 val accuracy 0.592
fold 2, epoch 20, val loss 130.93735945224762 val accuracy 0.6012
fold 2, epoch 40, val loss 154.3684406876564 val accuracy 0.6097
fold 2, epoch 60, val loss 155.02005279064178 val accuracy 0.5992
fold 2, epoch 80, val loss 151.43384152650833 val accuracy 0.6012
fold 2, epoch 100, val loss 161.47147917747498 val accuracy 0.6033
fold 2, epoch 120, val loss 172.37312197685242 val accuracy 0.5992
fold 3, epoch 20, val loss 137.87380999326706 val accuracy 0.5995
fold 3, epoch 40, val loss 172.58563655614853 val accuracy 0.6062
fold 3, epoch

[I 2023-11-22 21:49:29,347] Trial 6 finished with value: 0.5943999999999999 and parameters: {'lr': 0.0002571152733227137, 'l2_lambda': 4.291354366012778e-07, 'batch_size': 64, 'epochs': 120, 'flanking_out_channel1': 256, 'flanking_out_channel2': 128, 'upper_out_channel1': 512, 'upper_out_channel2': 128, 'lower_out_channel1': 128, 'lower_out_channel2': 128, 'concat_fc1_out': 512, 'concat_dropout_rate_fc1': 0, 'concat_fc2_out': 32, 'concat_dropout_rate_fc2': 0}. Best is trial 5 with value: 0.6523.


fold 3, epoch 120, val loss 195.59751397371292 val accuracy 0.592
fold 1, epoch 20, val loss 160.45536828041077 val accuracy 0.5842
fold 1, epoch 40, val loss 188.76232302188873 val accuracy 0.5902
fold 1, epoch 60, val loss 218.50530874729156 val accuracy 0.5893
fold 1, epoch 80, val loss 229.36381816864014 val accuracy 0.5865
fold 1, epoch 100, val loss 241.5366997718811 val accuracy 0.5908
fold 1, epoch 120, val loss 239.01858699321747 val accuracy 0.5762
fold 1, epoch 140, val loss 229.88340187072754 val accuracy 0.6007
